# MPI Matrix Multiplication example

In [11]:
%%writefile mpi_matmul.py
#!/usr/bin/env python
import numpy as np
from mpi4py import MPI
import math
 
def explicit_matmul(A,B):
    #A[m][n]
    #B[n][p]
    #C[m][p]   
    C_temp = [[0 for x in range(np.shape(A)[0])] for y in range(np.shape(B)[1])]
    for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
        for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
            for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
                C_temp[i][j] += A[i][k] * B[k][j]
    return(C_temp)
  
def explicit_matmul_mpi(A,B,rank,size):
    #A[m][n]
    #B[n][p]
    #C[m][p]
    C_temp = [[0 for x in range(np.shape(A)[0])] for y in range(np.shape(B)[1])]
    for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
        for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
            step = math.floor(np.shape(A)[1]/size)
            if (np.shape(A)[1]/size % 1 != 0) and (rank == size-1):
                for k in range(rank*step,np.shape(A)[1]):
                    C_temp[i][j] += A[i][k] * B[k][j]
            else:
                for k in range(rank*step,rank*step+step):
                    C_temp[i][j] += A[i][k] * B[k][j]
             
    return(C_temp)
 

t1_mpi = MPI.Wtime()
##################################################################
##################################################################
##################################################################


#1: initiate the communicator
comm = MPI.COMM_WORLD

#2: Initialize the arrays A,B... Since we are using np.random, they will not be the same per rank!!!
if comm.Get_rank() == 0:
    AX=AY=BX=BY=100
    A = np.random.rand(AX,AY)
    B = np.random.rand(BX,BY)
else:
    A = None
    B = None
    
#3: Broadcast them to the other ranks!
A = comm.bcast(A,root=0)
B = comm.bcast(B,root=0)
 
if comm.Get_rank() == 0:
    print("============================================================================")
    print("Performing Matrix Multiplication of two matricies of size (%d,%d)" % (AX,AY) )
    print("Using %d parallel MPI processes" % comm.Get_size())
     
#4: calc matrix multiply for each rank
result = explicit_matmul_mpi(A,B,comm.Get_rank(),comm.Get_size())
 
#5: "Gather" the results from the ranks and Make sure results are put back together and correctly!!!
### HINT: use np.sum! 
C_mpi_parallel = comm.gather(result)
C_mpi_parallel = np.sum(C_mpi_parallel,axis=0)

##################################################################
##################################################################
##################################################################
 
t2_mpi = MPI.Wtime()
 
if comm.Get_rank() == 0:
    print("============================================================================")
        
    t1_explicit = MPI.Wtime()
    C_explicit = explicit_matmul(A,B)
    t2_explicit = MPI.Wtime()
 
    t1_numpy = MPI.Wtime()
    C_np = np.matmul(A,B)
    t2_numpy = MPI.Wtime()
 
    if not np.allclose(C_explicit, C_mpi_parallel, rtol=1e-10, atol=1e-10):
        print("C_parallel is not equal to C_explicit!!")
    if not np.allclose(C_mpi_parallel, C_np, rtol=1e-10, atol=1e-10):
        print("C_parallel is not equal to C_np!!")
    if not np.allclose(C_explicit, C_np, rtol=1e-10, atol=1e-10):
        print("C_np is not equal to C_explicit!!")
     
    print("Performance=======")
    print("explicit: ",t2_explicit-t1_explicit)
    print("MPI: ",t2_mpi-t1_mpi)
    print("numpy matmul: ",t2_numpy-t1_numpy)

Overwriting mpi_matmul.py


In [12]:
!mpirun -np 3 --oversubscribe python3 mpi_matmul.py

Performing Matrix Multiplication of two matricies of size (100,100)
Using 3 parallel MPI processes
Performance=======
explicit:  0.591897371
MPI:  0.259363269
numpy matmul:  0.0013438660000000269


### submit it to the queue

In [ ]:
%%writefile mpi_jobscript.sh
#!/bin/bash

#SBATCH -n 96
#SBATCH -p normal
#SBATCH -t 00:30:00

module load 2021
module load SciPy-bundle/2021.05-foss-2021a

mpirun -np 96 --oversubscribe python3 mpi_matmult.py